<a href="https://colab.research.google.com/github/82KJ/ARS-system-for-protecting-telephone-counselor/blob/main/SH/Test_1_to_5/sample_classifier_combination_test_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 Setting

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689003 sha256=9d21b7d511a4c7c39c1108c867d703c08765f79df69be5dcf600c20025be8a69
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9x5f3uwn
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9x5f3uwn
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
device = torch.device("cuda:0")

In [14]:
# cd /content/drive/MyDrive/ARS-Protection/ARS-system-for-protecting-telephone-counselor/SH/Test_1_to_5
%cd /content/drive/MyDrive/bert

/content/drive/MyDrive/bert


In [15]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/drive/MyDrive/bert/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/drive/MyDrive/bert/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


## 데이터셋 전처리 진행

In [16]:
import pandas as pd
data1  = pd.read_csv("sample_data_5.csv")

In [17]:
data1.head()

,Unnamed: 0,0,1
0,0,형광색 바지 입는 급식들 왜 이렇게 많아?,0
1,1,ㅈ같은 유행 돌았나보네ㅋㅋ,1
2,2,형광 바지 입고 모여있는 급식무리 잡아서 패죽여야 하는데,2
3,3,자기 몸 긍정 운동가들이라고 하면서 꼴값 떠는 한녀들을 죽여 버리고 싶다.,2
4,4,자기 몸을 긍정하는 건 오히려 바람직한 거 아니야?,0


In [18]:
print(len(data1))

85354


In [19]:
data2 = pd.read_csv("sample_data_3.csv")

In [20]:
data2.head()

,Unnamed: 0,0,1
0,0,늙은 것들 중에도 유독 폐급들이 나이로 대접받고자 갑질함.,2
1,1,"늙어서 이룬 것 없이 가난하면, 부끄러워서라도 집에서 쥐죽은 듯 살다가 죽어야지",2
2,2,벌레 같은게 어딜 기어나와서 대접을 요구하나,2
3,3,"인생 쓰레기 같이 살아서 이룬 것 없는 것들이, 열등감에 가득차서 오히려 밖에서 대...",2
4,4,"외국이 좋은 게 노인이라도 밖에서 진상 부리면 막말 쳐 듣거나, 말 그대로 쳐 맞음.",2


In [21]:
print(len(data2))

64837


In [22]:
data = pd.concat([data1, data2],axis = 0)
#data1, data2 결합한 data 생성

In [23]:
data.head()

,Unnamed: 0,0,1
0,0,형광색 바지 입는 급식들 왜 이렇게 많아?,0
1,1,ㅈ같은 유행 돌았나보네ㅋㅋ,1
2,2,형광 바지 입고 모여있는 급식무리 잡아서 패죽여야 하는데,2
3,3,자기 몸 긍정 운동가들이라고 하면서 꼴값 떠는 한녀들을 죽여 버리고 싶다.,2
4,4,자기 몸을 긍정하는 건 오히려 바람직한 거 아니야?,0


In [24]:
print(len(data))

150191


In [25]:
data_list = list()
for sen, lab in zip(data["0"], data["1"]):
  data_list.append([sen,lab])

In [26]:
data_list[:10]

[['형광색 바지 입는 급식들 왜 이렇게 많아?', 0],
 ['ㅈ같은 유행 돌았나보네ㅋㅋ', 1],
 ['형광 바지 입고 모여있는 급식무리 잡아서 패죽여야 하는데', 2],
 ['자기 몸 긍정 운동가들이라고 하면서 꼴값 떠는 한녀들을 죽여 버리고 싶다.', 2],
 ['자기 몸을 긍정하는 건 오히려 바람직한 거 아니야?', 0],
 ['뚱뚱하다고 자기 몸을 혐오하는 건 바람직하지 않잖아.', 0],
 ['근데 자기 몸을 긍정한다고 뚱뚱한 걸 방치하는 경우도 있잖아.', 0],
 ['뚱뚱한 걸 방치하는 건 자기 건강을 해치는 행위지.', 0],
 ['난 아침에는 항상 시리얼을 먹어', 0],
 ['그딴 과자부스러기를 왜먹어?', 2]]

## Train data, Test data 분리

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
train_set, test_set = train_test_split(data_list, test_size=0.25, random_state=0) # train : test = 4 : 1

In [29]:
len(train_set), len(test_set)

(112643, 37548)

In [30]:
train_set[0]

['여자 관련 스캔들에 구설수…. 여기 어디냐?', 0]

## KoBERT 입력 데이터 구성하기
BERT 입력값은 (token + segment + position embedding의 합으로 구성)  
https://happy-obok.tistory.com/23 여기서 확인!!

In [31]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))
    

In [32]:
# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [33]:
# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/MyDrive/bert/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [34]:
data_train[0]

## 출력값 설명
## 1번 array - 1로 padding된 sequence - 총 길이 64 고정
## 2번 array - padding 전 sequnce 길이 - 42
## 3번 array - 어텐션 마스크 sequence (1로 padding이 된 부분은 어텐션 함수가 적용되지 않아도 됨을 알려주는 시퀀스)
##           - 일단은 전부 0으로 구성하고, 아래의 gen_attention_mask에서 필요한 부분만 1로 변경
## 마지막 숫자 - label값

(array([   2, 3318, 1083,  517, 6678, 6896, 1115, 6566, 6629,  517,   55,
         517,   54, 3301, 3223, 5689,  633,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [35]:
# Torch에 맞게 최종 입력 dataset 구성 

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBert 다중 분류기를 위한 미세 조정 모델 만들기


In [36]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=4, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 4 (label이 4개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

In [37]:
# Kobert + classifier 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer (Adam), scheduler 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [38]:
for e in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0
    model.train()

    # train set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # loss 는 CrossEntropyLoss를 이용 -> backpropagation 학습 진행
        loss = loss_fn(out, label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # train data set 정확도 확인
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    # test set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        # test set 정확도 확인
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-38-26ba33054b28>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1761 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.487223744392395 train acc 0.203125
epoch 1 batch id 201 loss 0.8936998844146729 train acc 0.4752021144278607
epoch 1 batch id 401 loss 0.7143015265464783 train acc 0.5736050498753117
epoch 1 batch id 601 loss 0.8663545250892639 train acc 0.6143926788685524
epoch 1 batch id 801 loss 0.8051759600639343 train acc 0.642478152309613
epoch 1 batch id 1001 loss 0.43524298071861267 train acc 0.6606986763236763
epoch 1 batch id 1201 loss 0.6477546095848083 train acc 0.6736053288925895
epoch 1 batch id 1401 loss 0.6071522831916809 train acc 0.6840426481084939
epoch 1 batch id 1601 loss 0.5979684591293335 train acc 0.693502108057464
epoch 1 train acc 0.7000638841567292


<ipython-input-38-26ba33054b28>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/587 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7805700209075421


  0%|          | 0/1761 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.15978367626667023 train acc 0.953125
epoch 5 batch id 201 loss 0.11254365742206573 train acc 0.916122512437811
epoch 5 batch id 401 loss 0.21937309205532074 train acc 0.9180174563591023
epoch 5 batch id 601 loss 0.3591267466545105 train acc 0.9186512063227953
epoch 5 batch id 801 loss 0.2275466024875641 train acc 0.920919163545568
epoch 5 batch id 1001 loss 0.08355914056301117 train acc 0.9233891108891109
epoch 5 batch id 1201 loss 0.33231794834136963 train acc 0.9240606786011657
epoch 5 batch id 1401 loss 0.19212155044078827 train acc 0.9248862419700214
epoch 5 batch id 1601 loss 0.16195400059223175 train acc 0.925710493441599
epoch 5 train acc 0.9264799829642248


  0%|          | 0/587 [00:00<?, ?it/s]

epoch 5 test acc 0.7778162265758092


## 학습된 Model로 결과 예측해보기

In [39]:
def predict(predict_sentence):

    # 1. data set 구성 (문장, 라벨)
    data = [predict_sentence, '0']
    dataset_another = [data]

    # 2. data를 bert의 입력에 맞게 변환하기
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # 모델 forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # torch out -> numpy 형식으로 변환
        test_eval=[]
        logits = out[0].detach().cpu().numpy()
        
        # 값이 가장 큰 인덱스를 출력
        if np.argmax(logits) == 0:
            test_eval= "일반"
        elif np.argmax(logits) == 1:
            test_eval = "욕설"
        elif np.argmax(logits) == 2:
            test_eval = "폭언"
        elif np.argmax(logits) == 3:
            test_eval = "성희롱"

        print(">> " + test_eval + " 문장입니다.")
     

In [42]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "q":
      break
    
    predict(sentence)
    print()
     

하고싶은 말을 입력해주세요 : 시발
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 씨발
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 시발, 장난치냐
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 시발 장난치냐
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 시발 너 뭐임
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 시발 너 뭐하냐
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : 씨발 장난치냐
>> 욕설 문장입니다.

하고싶은 말을 입력해주세요 : q


In [43]:
torch.save(model.state_dict(),'test3_5_dropout50_sibal.pth')


In [44]:
model.load_state_dict(torch.load('test3_5_dropout50_sibal.pth'))

<All keys matched successfully>